# Serverless Task Migration
This app is designed to help identify tasks running in your snowflake that are underutilizing a warehouse so that they can be changed to a serverless task. This is designed to migrate these tasks to improve the proficiency of these services as well as save on the cost of execution, as an under-utilized warehouse often charges for downtime when spun up for short tasks

This Notedbook will drop a streamlit file at SIT_SOLUTIONS.STM.CODE_STAGE and install the app SIT_SOLUTIONS.STM.STM_STREAMLIT.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import pickle
import codecs
import os
import base64

from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.sql(f"""ALTER SESSION SET QUERY_TAG = '{{"origin":"sf_sit","name":"sit_serveless_task_migration","version":{{"major":1, "minor":0}} }}'""").collect()

current_database = session.sql("SELECT CURRENT_DATABASE()").collect()[0][0]
current_schema = session.sql("SELECT CURRENT_schema()").collect()[0][0]
try:
    # session.sql("SELECT * from INSTALL_TEST.TEST.TEST_OPERATIONS").collect()
    st.session_state.opps_test_info = session.sql(f"SELECT * from {current_database}.{current_schema}.TEST_OPERATIONS").to_pandas()
    run_mode = "test"
except:
    st.success("Running Customer Install. Please follow the steps!")
    run_mode = "customer"

Please choose the location you'd like to install to

In [ ]:
if run_mode == 'test':
    opps_info = st.session_state.opps_test_info
    install_db = opps_info[opps_info["KEY"] == "INSTALL_DB"]["VALUE"].values[0]
    admin_schema = opps_info[opps_info["KEY"] == "ADMIN_SCHEMA"]["VALUE"].values[0]
    results_schema = opps_info[opps_info["KEY"] == "RESULT_SCHEMA"]["VALUE"].values[0]
    ops_schema = opps_info[opps_info["KEY"] == "OPS_SCHEMA"]["VALUE"].values[0]
    install_stage = opps_info[opps_info["KEY"] == "INSTALL_STAGE"]["VALUE"].values[0]
    ops_wh = opps_info[opps_info["KEY"] == "OPS_WH"]["VALUE"].values[0]
elif run_mode =="customer":
    install_db = st.text_input("Database")
    admin_schema = st.text_input("Admin Schema")
    results_schema = st.text_input("Results Schema")
    ops_schema = st.text_input("Operations Schema")
    install_stage = st.text_input("Install Stage (for hosting code files)")
    ops_wh = st.text_input("User Warehouse")

In [ ]:
CREATE OR REPLACE PROCEDURE {{install_db}}.{{admin_schema}}.PUT_TO_STAGE(STAGE VARCHAR, FILENAME VARCHAR, CONTENT VARCHAR)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION=3.8
PACKAGES=('snowflake-snowpark-python')
HANDLER='put_to_stage'
COMMENT='{"origin": "sf_sit","name": "sit_serveless_task_migration","version": "{major: 1, minor: 0}"}'
AS $$
import io
import os

def put_to_stage(session, stage, filename, content):
    local_path = '/tmp'
    local_file = os.path.join(local_path, filename)
    f = open(local_file, "w", encoding='utf-8')
    f.write(content)
    f.close()
    session.file.put(local_file, '@'+stage, auto_compress=False, overwrite=True)
    return "saved file "+filename+" in stage "+stage
$$;


In [ ]:
encoded_obj = """gASVgBcAAAAAAAB9lCiMBWZpbGVzlH2UKIwcc2VydmVybGVzc190YXNrX21pZ3JhdGlvbi5weZRC
5xYAACMgSW1wb3J0IHB5dGhvbiBwYWNrYWdlcwppbXBvcnQgc3RyZWFtbGl0IGFzIHN0CmltcG9y
dCBwYW5kYXMgYXMgcGQKaW1wb3J0IHRpbWUKZnJvbSBzbm93Zmxha2Uuc25vd3BhcmsuY29udGV4
dCBpbXBvcnQgZ2V0X2FjdGl2ZV9zZXNzaW9uCgpzdC5zZXRfcGFnZV9jb25maWcobGF5b3V0PSJ3
aWRlIikKIyBXcml0ZSBkaXJlY3RseSB0byB0aGUgYXBwCnN0LmltYWdlKCdodHRwczovL3Jhdy5n
aXRodWJ1c2VyY29udGVudC5jb20vU25vd2ZsYWtlLUxhYnMvZW1lcmdpbmctc29sdXRpb25zLXRv
b2xib3gvbWFpbi9iYW5uZXJfZW1lcmdpbmdfc29sdXRpb25zX3Rvb2xib3gucG5nJykKCnN0LnRp
dGxlKCJTZXJ2ZXJsZXNzIFRhc2tzIE1pZ3JhdGlvbiIpCiMgR2V0IHRoZSBjdXJyZW50IGNyZWRl
bnRpYWxzCnNlc3Npb24gPSBnZXRfYWN0aXZlX3Nlc3Npb24oKQoKY3NzID0gIiIiCjxzdHlsZT4K
ICAgIC8qVVNFUyBUSEUgSEFTIFNFTEVDVE9SIFRPIEZJTFRFUiBCWSBUSEUgTkVTVEVEIENPTlRB
SU5FUiovCgpkaXZbcm9sZT0iZGlhbG9nIl06aGFzKC5zdC1rZXktbWlncmF0ZV9jb250YWluZXIp
ewogICAgd2lkdGg6ODUlOwp9Cjwvc3R5bGU+IiIiCnN0Lmh0bWwoY3NzKQoKCkBzdC5kaWFsb2co
Ik1pZ3JhdGUgVGFzayIpCmRlZiBtaWdyYXRlX3Rhc2sodGFza19uYW1lLCB3aF9uYW1lLCB3aF9z
aXplKToKICAgIHdpdGggc3QuY29udGFpbmVyKGtleT0ibWlncmF0ZV9jb250YWluZXIiKToKICAg
ICAgICBzdC53cml0ZShmIlRoaXMgTWlncmF0aW9uIHdpbGwgcmVtb3ZlIHRhc2sge3Rhc2tfbmFt
ZX0iKQogICAgICAgIHN0LndyaXRlKGYiZnJvbSB3YXJlaG91c2Uge3doX25hbWV9LCBhbmQgbWFr
ZSBpdCBzZXJ2ZXJsZXNzIHdpdGggdGhlIGJlbG93IHNldHRpbmciKQogICAgICAgIHdoX3NpemVf
bGlzdCA9IFsiWFNNQUxMIiwiU01BTEwiLCJNRURJVU0iLCJMQVJHRSIsIlhMQVJHRSIsIlhYTEFS
R0UiLCJYWFhMQVJHRSJdCiAgICAgICAgd2hfc2l6ZSA9IHN0LnNlbGVjdGJveCgiSW5pdGlhbCBX
YXJlaG91c2UgU2l6ZSIsd2hfc2l6ZV9saXN0LGluZGV4PXdoX3NpemVfbGlzdC5pbmRleCh3aF9z
aXplLnJlcGxhY2UoJy0nLCcnKS51cHBlcigpKSkKICAgICAgICBfXyxjb2wyID0gc3QuY29sdW1u
cygoMTAsMikpCiAgICAgICAgaWYgY29sMi5idXR0b24oIk1pZ3JhdGUiLCB0eXBlPSJwcmltYXJ5
Iik6CiAgICAgICAgICAgIHNlc3Npb24uc3FsKGYiQUxURVIgVEFTSyB7dGFza19uYW1lfSBTVVNQ
RU5EIikuY29sbGVjdCgpCiAgICAgICAgICAgIHNlc3Npb24uc3FsKGYiQUxURVIgVEFTSyB7dGFz
a19uYW1lfSBVTlNFVCBXQVJFSE9VU0UiKS5jb2xsZWN0KCkKICAgICAgICAgICAgc2Vzc2lvbi5z
cWwoZiJBTFRFUiBUQVNLIHt0YXNrX25hbWV9IFNFVCBVU0VSX1RBU0tfTUFOQUdFRF9JTklUSUFM
X1dBUkVIT1VTRV9TSVpFID0gJ1hTTUFMTCc7IikuY29sbGVjdCgpCiAgICAgICAgICAgIHNlc3Np
b24uc3FsKGYiIiJBTFRFUiBUQVNLIHt0YXNrX25hbWV9IFNFVCBDT01NRU5UID0gJ3t7Im9yaWdp
biI6ICJzZl9zaXQiLCJuYW1lIjogInNpdF9zZXJ2ZWxlc3NfdGFza19taWdyYXRpb24iLCJ2ZXJz
aW9uIjogInt7bWFqb3I6IDEsIG1pbm9yOiAwfX0ifX0nIiIiKS5jb2xsZWN0KCkKICAgICAgICAg
ICAgc2Vzc2lvbi5zcWwoZiJBTFRFUiBUQVNLIHt0YXNrX25hbWV9IFJFU1VNRSIpLmNvbGxlY3Qo
KQogICAgICAgICAgICBzdC5zdWNjZXNzKGYie3Rhc2tfbmFtZX0gTWlncmF0ZWQhIikKICAgICAg
ICAgICAgdGltZS5zbGVlcCg1KQogICAgICAgICAgICBzdC5yZXJ1bigpCiAgICAgICAgICAgIAoK
QHN0LmNhY2hlX2RhdGEKZGVmIGdldF93YXJlaG91c2VzKHdoX25hbWVzKToKICAgIHJldHVybiBz
ZXNzaW9uLnNxbChmIiIiCiAgICBTRUxFQ1QgRElTVElOQ1QgV0FSRUhPVVNFX05BTUUsIENPTkNB
VCh0LkRBVEFCQVNFX05BTUUsJy4nLHQuU0NIRU1BX05BTUUsJy4nLCB0Lk5BTUUpIGFzIEZVTExf
TkFNRSwgdC5uYW1lIEZST00gU05PV0ZMQUtFLkFDQ09VTlRfVVNBR0UuVEFTS19ISVNUT1JZIHQK
ICBKT0lOIFNOT1dGTEFLRS5BQ0NPVU5UX1VTQUdFLlFVRVJZX0hJU1RPUlkgcSBPTiB0LlFVRVJZ
X0lEID0gcS5RVUVSWV9JRAogIFJJR0hUIEpPSU4gU05PV0ZMQUtFLkFDQ09VTlRfVVNBR0UuTUVU
RVJJTkdfSElTVE9SWSB3IE9OIHEud2FyZWhvdXNlX25hbWUgPSB3Lm5hbWUKICBXSEVSRSAKICBR
VUVSWV9TVEFSVF9USU1FIEJFVFdFRU4gdy5TVEFSVF9USU1FIEFORCB3LkVORF9USU1FCiAgQU5E
IE5PVCBDT05UQUlOUyhXQVJFSE9VU0VfTkFNRSwnQ09NUFVURV9TRVJWSUNFJykgCiAgQU5EIFND
SEVEVUxFRF9USU1FIEJFVFdFRU4gZGF0ZWFkZChEQVksIC0gMTQsIENVUlJFTlRfVElNRVNUQU1Q
KCkpIEFORCBDVVJSRU5UX1RJTUVTVEFNUCgpCiAgQU5EIHcubmFtZSBpbiAoe3doX25hbWVzfSk7
CiAgICAiIiIpLnRvX3BhbmRhcygpCgppZiAicXVhbF90YXNrcyIgbm90IGluIHN0LnNlc3Npb25f
c3RhdGU6CiAgICB3aXRoIHN0LnNwaW5uZXIoIkZldGNoaW5nIFRhc2tzIik6CgogICAgICAgIHN0
LnNlc3Npb25fc3RhdGUucXVhbF90YXNrcyA9IHNlc3Npb24uc3FsKCIiIlNFTEVDVCBESVNUSU5D
VCBDT05DQVQodC5EQVRBQkFTRV9OQU1FLCcuJyx0LlNDSEVNQV9OQU1FLCcuJywgTkFNRSkgYXMg
RlVMTF9OQU1FLE5BTUUsIHQuUVVFUllfVEVYVCwgV0FSRUhPVVNFX05BTUUsIFdBUkVIT1VTRV9T
SVpFLCBpZmYoREFURURJRkYoIk1JTlVURSIsUVVFUllfU1RBUlRfVElNRSwgQ09NUExFVEVEX1RJ
TUUpIDwgMSwgJ1RSVUUnLCdGQUxTRScpIGFzIFNIT1JUX1JVTlRJTUUsIGlmZihEQVRFRElGRigi
TUlOVVRFIixRVUVSWV9TVEFSVF9USU1FLCBTQ0hFRFVMRURfVElNRSkgPiAxLCAnVFJVRScsJ0ZB
TFNFJykgYXMgUkFOX09WRVIgRlJPTSBTTk9XRkxBS0UuQUNDT1VOVF9VU0FHRS5UQVNLX0hJU1RP
UlkgdAogICAgICAgICAgSk9JTiBTTk9XRkxBS0UuQUNDT1VOVF9VU0FHRS5RVUVSWV9ISVNUT1JZ
IHEgT04gdC5RVUVSWV9JRCA9IHEuUVVFUllfSUQKICAgICAgICAgIFdIRVJFIAogICAgICAgICAg
KERBVEVESUZGKCJNSU5VVEUiLFFVRVJZX1NUQVJUX1RJTUUsIENPTVBMRVRFRF9USU1FKSA8IDEK
ICAgICAgICAgIE9SCiAgICAgICAgICBEQVRFRElGRigiTUlOVVRFIixRVUVSWV9TVEFSVF9USU1F
LCBTQ0hFRFVMRURfVElNRSkgPiAxKQogICAgICAgICAgQU5EIE5PVCBDT05UQUlOUyhXQVJFSE9V
U0VfTkFNRSwnQ09NUFVURV9TRVJWSUNFJykgCiAgICAgICAgICBBTkQgU0NIRURVTEVEX1RJTUUg
QkVUV0VFTiBkYXRlYWRkKERBWSwgLSAxNCwgQ1VSUkVOVF9USU1FU1RBTVAoKSkgQU5EIENVUlJF
TlRfVElNRVNUQU1QKCk7IiIiKS50b19wYW5kYXMoKQoKcXVhbF90YXNrcyA9IHN0LnNlc3Npb25f
c3RhdGUucXVhbF90YXNrcwoKd2l0aCBzdC5jb250YWluZXIoYm9yZGVyPVRydWUpOgogICAgc3Qu
d3JpdGUoIiIiVGhlIHRhc2tzIGJlbG93IGFyZSB0aGUgZmlyc3QgZ2xhbmNlIG9mIHRhc2tzIHRo
YXQgbWF5IHF1YWxpZnkgZm9yIG1pZ3JhdGlvbiB0byBzZXJ2ZXJsZXNzIHRhc2tzIFxuCiAgICBU
aGV5IGVpdGhlciBydW4gZm9yIHVuZGVyIGEgbWludXRlLCBvciBydW4gbG9uZ2VyIHRoYW4gdGhl
IHRpbWUgYmV0d2VlbiB0aGVpciBzY2hlZHVsZWQgcnVuIGFuZCB0aGUgbmV4dCBydW4KICAgICIi
IikKICAgIHN0LmRhdGFmcmFtZShxdWFsX3Rhc2tzLCB1c2VfY29udGFpbmVyX3dpZHRoPVRydWUs
IGNvbHVtbl9jb25maWc9eydGVUxMX05BTUUnOk5vbmV9KQoKd2l0aCBzdC5jb250YWluZXIoYm9y
ZGVyPVRydWUpOgogICAgc3Qud3JpdGUoIiIiVGhlIFRhc2tzIHRoYXQgcnVuIGxvbmcgaW4gdGhl
IHByZXZpb3VzIGxpc3QgYXJlIGdyZWF0IGNhbmRpZGF0ZXMgZm9yIHNlcnZlcmxlc3Mgd2FyZWhv
dXNlcyBhcyB0aGV5IGNhbiBoYXZlIGEgdGFyZ2V0IGZpbmlzaCBzZXQgYW5kIHRoZSB3YXJlaG91
c2Ugd2lsbCBzY2FsZSBmb3IgdGhlIHNjaGVkdWxlIiIiKQogICAgc3Qud3JpdGUoIiIiRm9yIHNo
b3J0IHJ1bm5pbmcgdGFza3MsIHRvIGZ1cnRoZXIgcXVhbGlmeSB0aGVtIHdlJ2xsIGNoZWNrIGhv
dyBtYW55IHRhc2tzIGdlbmVyYWxseSBydW4gb24gdGhlIHNhbWUgd2FyZWhvdXNlIiIiKQoKd2l0
aCBzdC5jb250YWluZXIoYm9yZGVyPVRydWUpOgogICAgc3Qud3JpdGUoIiIiVGhlIGJlbG93IGxp
c3Qgc2hvd3MgdGhlIG51bWJlciBvZiB0YXNrcyB0aGF0IGFsbCBydW4gYXJvdW5kIHRoZSBzYW1l
IHRpbWUgb24gZWFjaCB3YXJlaG91c2UiIiIpCgogICAgd2FyZWhvdXNlcyA9IHNldChxdWFsX3Rh
c2tzWyJXQVJFSE9VU0VfTkFNRSJdLnRvbGlzdCgpKQogICAgd2FyZWhvdXNlc19sID0gIid7fSci
LmZvcm1hdCgiJywgJyIuam9pbih3YXJlaG91c2VzKSkKCiAgICB3YXJlaG91c2VfbGlzdCA9IGdl
dF93YXJlaG91c2VzKHdhcmVob3VzZXNfbCkKICAgIHN0LmRhdGFmcmFtZSh3YXJlaG91c2VfbGlz
dCx1c2VfY29udGFpbmVyX3dpZHRoPVRydWUpCgogICAgc3Qud3JpdGUoIkZvciBhbnkgdGFza3Mg
dGhhdCBydW4gZm9yIGxlc3MgdGhhbiBhIG1pbnV0ZSBvbiBhIHdhcmVob3VzZSBhbG9uZSBzaG91
bGQgYmUgbWlncmF0ZWQgdG8gYSBzZXJ2ZXJsZXNzIHdhcmVob3VzZSIpCiAgICBzdC53cml0ZSgi
QWRkaXRpb25hbGx5IGFueSB0YXNrcyBzY2hlZHVsZWQgd2l0aCBhIHRvdGFsIHJ1bnRpbWUgb2Yg
bGVzcyB0aGFuIGEgZmV3IG1pbnV0ZXMgd291bGQgbGlrZWx5IGVhY2ggYmVuZWZpdCBmcm9tIGJl
aW5nIHNlcnZlcmxlc3MiKQogICAgc3Qud3JpdGUoIlJldmlldyBlYWNoIHdhcmVob3VzZSBiZWxv
dyBhbmQgbWlncmF0ZSBhbnkgdGFza3MgdGhhdCBhcmUgcmVjb21tZW5kZWQgb3IgdGhhdCB5b3Ug
d291bGQgbGlrZSB0byIpCiAgICAKICAgIGZvciB3aF9uYW1lIGluIHdhcmVob3VzZXM6CiAgICAg
ICAgd2hfdGFza3MgPSB3YXJlaG91c2VfbGlzdFt3YXJlaG91c2VfbGlzdFsiV0FSRUhPVVNFX05B
TUUiXSA9PSB3aF9uYW1lXVsiTkFNRSJdLnRvbGlzdCgpCiAgICAgICAgd2hfc2l6ZSA9IHF1YWxf
dGFza3NbcXVhbF90YXNrc1siV0FSRUhPVVNFX05BTUUiXSA9PSB3aF9uYW1lXVsiV0FSRUhPVVNF
X1NJWkUiXS52YWx1ZXNbMF0KICAgICAgICB3aXRoIHN0LmV4cGFuZGVyKHdoX25hbWUpOgogICAg
ICAgICAgICBzdC53cml0ZSgiIyMjIikKICAgICAgICAgICAgY29sMSxfXyxjb2wyID0gc3QuY29s
dW1ucygzKQogICAgICAgICAgICBjb2wxLnN1YmhlYWRlcigiKlRhc2sgTmFtZSoiKQoKICAgICAg
ICAgICAgZm9yIHdoX3Rhc2sgaW4gd2hfdGFza3M6CiAgICAgICAgICAgICAgICBmdWxsX3Rhc2tf
bmFtZSA9IHdhcmVob3VzZV9saXN0W3dhcmVob3VzZV9saXN0WyJOQU1FIl0gPT0gd2hfdGFza11b
IkZVTExfTkFNRSJdLnZhbHVlc1swXQogICAgICAgICAgICAgICAgZGVzY3JpYmVfdGFzayA9IHBk
LkRhdGFGcmFtZShzZXNzaW9uLnNxbChmIkRFU0NSSUJFIFRBU0sge2Z1bGxfdGFza19uYW1lfSIp
LmNvbGxlY3QoKSkKICAgICAgICAgICAgICAgIGlmIGRlc2NyaWJlX3Rhc2tbIndhcmVob3VzZSJd
LnZhbHVlc1swXToKICAgICAgICAgICAgICAgICAgICBzdC5kaXZpZGVyKCkKICAgICAgICAgICAg
ICAgICAgICBjb2wxLF9fLGNvbDIgPSBzdC5jb2x1bW5zKDMpCiAgICAgICAgICAgICAgICAgICAg
Y29sMS53cml0ZSh3aF90YXNrKQogICAgICAgICAgICAgICAgICAgIGNvbDIuYnV0dG9uKCJNaWdy
YXRlIixvbl9jbGljaz1taWdyYXRlX3Rhc2ssIGFyZ3M9KGZ1bGxfdGFza19uYW1lLHdoX25hbWUs
d2hfc2l6ZSksIGtleT13aF90YXNrKyJtaWdyYXRlX2J1dHRvbiIpCiAgICAgICAgCiAgICCUjA9l
bnZpcm9ubWVudC55bWyUQ0NuYW1lOiBzZl9lbnYKY2hhbm5lbHM6Ci0gc25vd2ZsYWtlCmRlcGVu
ZGVuY2llczoKLSBzdHJlYW1saXQ9MS4zOS4wlHWMBmltYWdlc5R9lHUu"""

In [ ]:
import pickle
import codecs
import os
import base64

from snowflake.snowpark.context import get_active_session
session = get_active_session()

def decode(obj):
    pickled = codecs.decode(obj.encode(), 'base64')
    return pickle.loads(pickled)


# Decode object to confirm it is the same as the original object
decoded_obj = decode(encoded_obj)

# Stage each file
for file_name, file_content in decoded_obj['files'].items():
    # replace double dollars to help with SQL compatibility
    file_content = str(file_content.decode("utf-8")).replace("$$", '"""')
    sql = """call {db}.{schema}.put_to_stage('{db}.{schema}.{stage}','{file_name}', $${file_content}$$)""".format(
        file_name=file_name, 
        file_content=file_content,
        db=install_db,
        schema=admin_schema,
        stage=install_stage)
    session.sql(sql).collect()

# Stage each image

In [ ]:
CREATE OR REPLACE STREAMLIT {{install_db}}.{{ops_schema}}.STM_STREAMLIT
ROOT_LOCATION = '@{{install_db}}.{{ops_schema}}.{{install_stage}}'
MAIN_FILE = 'serverless_task_migration.py'
QUERY_WAREHOUSE = {{ops_wh}}
COMMENT='{"origin": "sf_sit","name": "sit_serveless_task_migration","version": "{major: 1, minor: 0}"}';